# Perform Experiments with DeepFace on LFW dataset

In [1]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=4519f9991f1d7f973155d1870b05bc41e432fcc52eaa14e1c77ea4e85edb3d33
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [2]:
# built-in dependencies
import os

# 3rd party dependencies
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_lfw_pairs
from deepface import DeepFace

25-05-18 16:11:36 - Directory /root/.deepface has been created
25-05-18 16:11:36 - Directory /root/.deepface/weights has been created


In [3]:
print(f"This experiment is done with pip package of deepface with {DeepFace.__version__} version")

This experiment is done with pip package of deepface with 0.0.93 version


### Configuration Sets

In [4]:
# all configuration alternatives for 4 dimensions of arguments
alignment = [True, False]

# models = ["Facenet512", "Facenet", "VGG-Face", "ArcFace", "Dlib", "GhostFaceNet", "SFace", "OpenFace", "DeepFace", "DeepID"]
models = ["ArcFace", "SFace"]

# detectors = ["retinaface", "mtcnn", "fastmtcnn", "dlib", "yolov8", "yunet", "centerface", "mediapipe", "ssd", "opencv", "skip"]
detectors = ["yunet"]

# metrics = ["euclidean", "euclidean_l2", "cosine"]
metrics = ["cosine"]

expand_percentage = 0

### Create Required Folders if necessary

In [5]:
target_paths = ["lfwe", "dataset", "outputs", "outputs/test", "results"]
for target_path in target_paths:
    if not os.path.exists(target_path):
        os.mkdir(target_path)
        print(f"{target_path} dir is just created")

lfwe dir is just created
dataset dir is just created
outputs dir is just created
outputs/test dir is just created
results dir is just created


### Load LFW Dataset

In [6]:
pairs_touch = "outputs/test_lfwe.txt"
instances = 1000 #pairs.shape[0]

In [1]:
target_path = "dataset/test_lfw.npy"
labels_path = "dataset/test_labels.npy"

if os.path.exists(target_path) != True:
    fetch_lfw_pairs = fetch_lfw_pairs(subset = 'test', color = True
                                  , resize = 2
                                  , funneled = False
                                  , slice_= None
                                 )
    pairs = fetch_lfw_pairs.pairs
    labels = fetch_lfw_pairs.target
    target_names = fetch_lfw_pairs.target_names
    np.save(target_path, pairs)
    np.save(labels_path, labels)
else:
    if not os.path.exists(pairs_touch):
        # loading pairs takes some time. but if we extract these pairs as image, no need to load it anymore
        pairs = np.load(target_path)
    labels = np.load(labels_path)

NameError: name 'os' is not defined

In [ ]:
print(pairs.shape)
print(labels.shape)
print(target_names.shape)

In [ ]:
print(labels)

### Save LFW image pairs into file system

In [ ]:
# Create the target directory if it doesn't exist
target_dir = "lfwe/test"
os.makedirs(target_dir, exist_ok=True)

for i in tqdm(range(0, instances)):
    img1_target = os.path.join(target_dir, f"{i}_1.jpg")
    img2_target = os.path.join(target_dir, f"{i}_2.jpg")

    if not os.path.exists(img1_target):
        img1 = pairs[i][0]
        plt.imsave(img1_target, img1)

    if not os.path.exists(img2_target):
        img2 = pairs[i][1]
        plt.imsave(img2_target, img2)

# Create the pairs_touch file if needed
if not os.path.exists(pairs_touch):
    open(pairs_touch, 'a').close()

### Perform Experiments

This block will save the experiments results in outputs folder

In [ ]:
for model_name in models:
    for detector_backend in detectors:
        for distance_metric in metrics:
            for align in alignment:

                if detector_backend == "skip" and align is True:
                    # Alignment is not possible for a skipped detector configuration
                    continue

                alignment_text = "aligned" if align is True else "unaligned"
                task = f"{model_name}_{detector_backend}_{distance_metric}_{alignment_text}"
                output_file = f"outputs/test/{task}.csv"
                if os.path.exists(output_file):
                     #print(f"{output_file} is available already")
                     continue

                distances = []
                for i in tqdm(range(0, instances), desc = task):
                    img1_target = f"lfwe/test/{i}_1.jpg"
                    img2_target = f"lfwe/test/{i}_2.jpg"
                    result = DeepFace.verify(
                        img1_path=img1_target,
                        img2_path=img2_target,
                        model_name=model_name,
                        detector_backend=detector_backend,
                        distance_metric=distance_metric,
                        align=align,
                        enforce_detection=False,
                        expand_percentage=expand_percentage,
                    )
                    distance = result["distance"]
                    distances.append(distance)
                # -----------------------------------
                df = pd.DataFrame(list(labels), columns = ["actuals"])
                df["distances"] = distances
                df.to_csv(output_file, index=False)

### Calculate Results

Experiments were responsible for calculating distances. We will calculate the best accuracy scores in this block.

In [ ]:
data = [[0 for _ in range(len(models))] for _ in range(len(detectors))]
base_df = pd.DataFrame(data, columns=models, index=detectors)

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix

In [ ]:
for is_aligned in alignment:
    for distance_metric in metrics:

        current_df = base_df.copy()

        target_file = f"results/pivot_{distance_metric}_with_alignment_{is_aligned}.csv"
        # if os.path.exists(target_file):
        #     continue

        for model_name in models:
            for detector_backend in detectors:

                align = "aligned" if is_aligned is True else "unaligned"

                if detector_backend == "skip" and is_aligned is True:
                    # Alignment is not possible for a skipped detector configuration
                    align = "unaligned"

                source_file = f"outputs/test/{model_name}_{detector_backend}_{distance_metric}_{align}.csv"
                df = pd.read_csv(source_file)

                positive_mean = df[(df["actuals"] == True) | (df["actuals"] == 1)]["distances"].mean()
                negative_mean = df[(df["actuals"] == False) | (df["actuals"] == 0)]["distances"].mean()

                distances = sorted(df["distances"].values.tolist())

                # Collect accuracies for different distance thresholds
                items = []
                for i, distance in enumerate(distances):
                    if distance >= positive_mean and distance <= negative_mean:
                        sandbox_df = df.copy()
                        sandbox_df["predictions"] = False
                        idx = sandbox_df[sandbox_df["distances"] < distance].index
                        sandbox_df.loc[idx, "predictions"] = True

                        # Create directory if it doesn't exist
                        misclassified_dir = "outputs/misclassified"
                        os.makedirs(misclassified_dir, exist_ok=True)

                        # Save misclassified samples to a CSV file
                        misclassified_file = f"{misclassified_dir}/{model_name}_{detector_backend}_{distance_metric}_{align}_misclassified.csv"
                        misclassified_samples = sandbox_df[sandbox_df["actuals"] != sandbox_df["predictions"]]
                        misclassified_samples.to_csv(misclassified_file, index=False)

                        actuals = sandbox_df.actuals.values.tolist()
                        predictions = sandbox_df.predictions.values.tolist()
                        accuracy = 100*accuracy_score(actuals, predictions)
                        items.append((distance, accuracy))



                # Get best distance threshold for accuracy
                pivot_df = pd.DataFrame(items, columns = ["distance", "accuracy"])
                pivot_df = pivot_df.sort_values(by = ["accuracy"], ascending = False)
                threshold = pivot_df.iloc[0]["distance"]
                # print(f"threshold for {model_name}/{detector_backend} is {threshold}")
                accuracy = pivot_df.iloc[0]["accuracy"]

                # print(source_file, round(accuracy, 1))
                current_df.at[detector_backend, model_name] = round(accuracy, 1)
                current_df.at[detector_backend, 'threshold'] = threshold

        current_df.to_csv(target_file)
        print(f"{target_file} saved")

# Evaluate Results

In [ ]:
import pandas as pd
from IPython.display import display, HTML
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
distance_metrics = ["cosine"] # re-define to prevent collision w/sklearn.metrics

# Main results

In [ ]:
for align in alignment:
    for metric in distance_metrics:
        df = pd.read_csv(f"results/pivot_{metric}_with_alignment_{align}.csv")
        df = df.rename(columns = {'Unnamed: 0': 'detector'})
        df = df.set_index('detector')

        print(f"{metric} for alignment {align}")
        display(HTML(df.to_html()))
        display(HTML("<hr>"))

In [ ]:
def create_github_table():
    for metric in distance_metrics:
        for align in [True, False]:
            df = pd.read_csv(f"results/pivot_{metric}_with_alignment_{align}.csv")
            df = df.rename(columns = {'Unnamed: 0': 'detector'})
            df = df.set_index('detector')

            print(f"Performance Matrix for {metric} while alignment is {align} \n")
            header = "| | "
            for col_name in df.columns.tolist():
                header += f"{col_name} |"
            print(header)
            # -------------------------------
            seperator = "| --- | "
            for col_name in df.columns.tolist():
                seperator += " --- |"
            print(seperator)
            # -------------------------------
            for index, instance in df.iterrows():
                line = f"| {instance.name} |"
                for i in instance.values:
                    if i < 97.5:
                        line += f"{i} |"
                    else:
                        line += f"**{i}** |"
                print(line)

            print("\n---------------------------")

In [ ]:
# create_github_table()

# Alignment impact

In [ ]:
align_df = None

for distance_metric in distance_metrics:
    df1 = (
        pd.read_csv(f"results/pivot_{distance_metric}_with_alignment_True.csv")
        .rename(columns = {'Unnamed: 0': 'detector'})
        .set_index('detector')
    )
    df2 = (
        pd.read_csv(f"results/pivot_{distance_metric}_with_alignment_False.csv")
        .rename(columns = {'Unnamed: 0': 'detector'})
        .set_index('detector')
    )
    df1 = df1[df1.index != "skip"]
    df2 = df2[df2.index != "skip"]
    pivot_df = df1.subtract(df2)

    pivot_df = pivot_df.max()
    pivot_df = pd.DataFrame(pivot_df, columns=[f'alignment_impact_of_{distance_metric}'])
    # display(HTML(pivot_df.to_html()))

    if align_df is None:
        align_df = pivot_df.copy()
    else:
        align_df = align_df.merge(pivot_df, left_index=True, right_index=True)

# display(HTML(align_df.to_html()))
align_df = pd.DataFrame(align_df.max(axis=1), columns = ["max_alignment_impact"])
align_df = align_df.sort_values(by=["max_alignment_impact"], ascending=False)
display(HTML(align_df.to_html()))

# facial recognition model's best scores

In [ ]:
df = pd.DataFrame()
for align in alignment:
    for distance_metric in distance_metrics:
        tmp_df = (
            pd.read_csv(f"results/pivot_{distance_metric}_with_alignment_{align}.csv")
            .rename(columns = {'Unnamed: 0': 'detector'})
            .set_index('detector')
        )
        df = pd.concat([df, tmp_df])

pivot_df = pd.DataFrame(df.max(), columns = ["best_accuracy_score"])

# add human comparison
pivot_df.loc["Human-beings"] = 97.5

pivot_df = pivot_df.sort_values(by = ["best_accuracy_score"], ascending = False)
pivot_df

# ROC Curves

In [ ]:
def plot_roc(model_name, detector_backend, distance_metric, align):
    alignment_text = "aligned" if align == True else "unaligned"

    df = pd.read_csv(f"outputs/test/{model_name}_{detector_backend}_{distance_metric}_{alignment_text}.csv")

    #normalize
    df["distances_normalized"] = df["distances"] / df["distances"].max()
    df["actuals_normalized"] = 0
    idx = df[df["actuals"] == False].index
    df.loc[idx, "actuals_normalized"] = 1

    y_actual = df["actuals_normalized"].values.tolist()
    y_pred_proba = df["distances_normalized"].values.tolist()
    y_pred_proba_non_normalized = df["distances"].values.tolist()

    fpr, tpr, _ = metrics.roc_curve(y_actual, y_pred_proba)

    # Convert probabilities to binary predictions using distance threshold
    th = 0.752753 if not align else 0.752063
    y_pred = [1 if prob > th else 0 for prob in y_pred_proba_non_normalized]

    # Calculate F1-score
    f1 = metrics.f1_score(y_actual, y_pred)
    f1 = round(f1, 4)

    # Calculate confusion matrix
    cm = metrics.confusion_matrix(y_actual, y_pred)

    # Plot confusion matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(f"Confusion Matrix\nModel: {model_name}, Detector: {detector_backend}, Distance Metric: {distance_metric}, Alignment: {alignment_text}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    auc = metrics.roc_auc_score(y_actual, y_pred_proba)
    auc = round(auc, 4)

    # best accuracy score
    result_path = f"results/pivot_{distance_metric}_with_alignment_{align}.csv"
    result_df = pd.read_csv(result_path)
    acc = result_df[result_df["Unnamed: 0"] == detector_backend][model_name].values[0]

    label = f"{model_name}_{detector_backend}_{distance_metric}_{alignment_text} (acc: {acc}, auc: {auc}, f1: {f1})"

    return acc, auc, fpr, tpr, f1, label

In [ ]:
# to show all models in same graph
plt.figure(figsize=(17, 8))

for model_name in models:
    # to show graphs model by model
    # plt.figure(figsize=(17, 8))
    accs = []
    aucs = []
    fprs = []
    tprs = []
    labels = []
    f1s = []
    for distance_metric in distance_metrics:
        # for detector_backend in robust_face_detectors:
        for detector_backend in detectors:
            for align in alignment:
                if detector_backend == "skip" and align is True:
                    continue
                acc, auc, fpr, tpr, f1, label = plot_roc(model_name, detector_backend, distance_metric, align)
                accs.append(acc)
                aucs.append(auc)
                fprs.append(fpr)
                tprs.append(tpr)
                f1s.append(f1)
                labels.append(label)
    # ---------------------------------
    #sort by auc
    df = pd.DataFrame({"acc": accs, "auc": aucs, "fpr": fprs, "tpr": tprs, "f1": f1s, "label": labels})
    # df = df.sort_values(by = ["auc"], ascending = False).reset_index()
    df = df.sort_values(by = ["acc"], ascending = False).reset_index()

    for index, instance in df.iterrows():
        fpr = instance["fpr"]
        tpr = instance["tpr"]
        auc = instance["auc"]
        acc = instance["acc"]
        f1 = instance["f1"]
        label = instance["label"]

        plt.plot(fpr, tpr, label=label)
        plt.ylabel("True Positive Rate")
        plt.xlabel("False Positive Rate")
        plt.legend(loc="lower center", ncol=2)
        # normally this should be [0, 1] but that scale makes graphs not legible
        # plt.xlim([0, 1])
        plt.xlim([0, 0.3])

        # to show the best auc value
        break

    # to show graphs model by model
    # plt.show()
    # print("----------------")

# to show all models in same graph
plt.show()

# Misclassified

In [ ]:
!ls "outputs/misclassified"

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

# Define the directory containing misclassified samples
misclassified_dir = "outputs/misclassified"

# Define the base directory where images are stored
image_base_dir = "lfwe/test"

# Iterate over each CSV file in the misclassified directory
for filename in os.listdir(misclassified_dir):
    if filename.endswith("_misclassified.csv"):
        # Extract model_name, detector_backend, distance_metric, and align from the filename
        parts = filename.split('_')
        model_name = parts[0]
        detector_backend = parts[1]
        distance_metric = parts[2]
        align = parts[3]

        # Read the CSV file
        misclassified_file = os.path.join(misclassified_dir, filename)
        misclassified_samples = pd.read_csv(misclassified_file)

        # Assuming the CSV contains filenames or indices that correspond to the image filenames
        # Display two random images for each group
        random_indices = random.sample(range(len(misclassified_samples)), min(2, len(misclassified_samples)))

        for i, sample_index in enumerate(random_indices):
            # Construct the image paths based on the naming convention
            image_path_1 = os.path.join(image_base_dir, f"{sample_index}_1.jpg")
            image_path_2 = os.path.join(image_base_dir, f"{sample_index}_2.jpg")

            # Display the images if they exist
            for image_path in [image_path_1, image_path_2]:
                if os.path.exists(image_path):
                    img = mpimg.imread(image_path)
                    plt.figure()
                    plt.imshow(img)
                    plt.title(f"Model: {model_name}, Detector: {detector_backend}, Distance Metric: {distance_metric}, Alignment: {align}\nRandom Image {i+1}")
                    plt.axis('off')
                    plt.show()
                else:
                    print(f"Image not found: {image_path}")
